<a href="https://colab.research.google.com/github/desireesosa/EDPII/blob/main/ElementoFinito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MÉTODO DEL ELEMENTO FINITO**

Es una técnica numérica general para obtener soluciones aproximadas de problemas de contorno gobernados por ecuaciones diferenciales. Este método se basa en la formulación variacional del problema y puede considerarse como una extensión natural de los métodos variacionales estudiados previamente.

***Discretización del dominio***

El intervalo del problema se subdivide en un número finito de subintervalos:

$$
[a,b] = \bigcup_{e=1}^{n_e} [x_e, x_{e+1}]
$$

***Aproximación local de la solución:***

La solución aproximada global puede escribirse como:
$$
u_h(x) = \sum_{i=1}^{N} U_i \, \phi_i(x)
$$

***Funciones de forma:***
Las funciones de forma tienen soporte local, es decir, cada función es distinta de cero solo en un número limitado de elementos. Se eligen de manera que:
$$
\phi_i(x_j) = \delta_{ij}
$$


***Ensamblaje del sistema global***

El sistema algebraico global se obtiene mediante el ensamblaje de las contribuciones de cada elemento. Cada elemento aporta una matriz elemental y un vector elemental, que se combinan siguiendo la conectividad de la malla.

$$
\mathbf{K}\mathbf{U} = \mathbf{F}
$$



*El método de los elementos finitos puede interpretarse como una aplicación sistemática del método de Galerkin sobre subdominios. En este sentido, FEM no es un método completamente distinto, sino una forma estructurada de aplicar métodos variacionales utilizando funciones definidas por partes.*

**Solución Problema 5.6.9**

Resolver por medio del metodo de elementos finitos:


$
-u''(x) + u(x) = x, \quad 0 < x < 1
$$



con condiciones de frontera esenciales:

$$
u(0) = 0, \quad u(1) = 0
$$



La solución aproximada se busca en la forma

$$
u_N(x) = \sum_{j=1}^{N} \alpha_j \, \beta_j(x),
$$

y la función de prueba se toma como

$$
v(x) = \sum_{i=1}^{N} \lambda_i \, \beta_i(x),
$$

donde las funciones base satisfacen

$$
\beta_i(0) = \beta_i(1) = 0.
$$

El sistema lineal resultante es

$$
\mathbf{K}\boldsymbol{\alpha} = \mathbf{F},
$$

con

$$
K_{ij} = \int_0^1 \left( \beta_i'(x)\beta_j'(x) + \beta_i(x)\beta_j(x) \right)\,dx,
\quad
F_i = \int_0^1 x\,\beta_i(x)\,dx.
$$


In [ ]:
import numpy as np
import sympy as sp
import pandas as pd

x = sp.symbols('x')
h = sp.Rational(1,4)

phi1 = (h - x)/h
phi2 = x/h

dphi1 = sp.diff(phi1, x)
dphi2 = sp.diff(phi2, x)

Kloc = sp.Matrix([
    [sp.integrate(dphi1*dphi1 + phi1*phi1, (x, 0, h)),
     sp.integrate(dphi1*dphi2 + phi1*phi2, (x, 0, h))],
    [sp.integrate(dphi2*dphi1 + phi2*phi1, (x, 0, h)),
     sp.integrate(dphi2*dphi2 + phi2*phi2, (x, 0, h))]
])

Kloc = np.array(Kloc, dtype=float)

K = np.zeros((3,3))

K[0,0] += Kloc[1,1]
K[1,1] += Kloc[0,0] + Kloc[1,1]
K[2,2] += Kloc[0,0]

K[0,1] += Kloc[0,1]
K[1,0] += Kloc[1,0]
K[1,2] += Kloc[0,1]
K[2,1] += Kloc[1,0]

f1 = sp.integrate(x*phi2.subs(x, x), (x, 0, h))
f2 = sp.integrate(x*(phi1.subs(x, x-h)), (x, h, 2*h))
f3 = sp.integrate(x*(phi2.subs(x, x-2*h)), (x, 2*h, 3*h))

F = np.array([f1, f2+f1, f3], dtype=float)

u = np.linalg.solve(K, F)

u1, u2, u3 = u

print(f"u1 = {u1:.4f}")
print(f"u2 = {u2:.4f}")
print(f"u3 = {u3:.4f}")

def u_exact(x):
    return (2*np.cos(1 - x) - np.sin(x))/np.cos(1) + x - 2

xv = np.array([0.0, 0.25, 0.5, 0.75, 1.0])
uv = np.array([0.0, u1, u2, u3, 1.0])

tabla = pd.DataFrame({
    "x": xv,
    "u_n": np.round(uv, 4),
    "u_exact": np.round(u_exact(xv), 4)
})

print(tabla)


u1 = 0.3270
u2 = 0.3320
u3 = 0.3423
      x     u_n  u_exact
0  0.00  0.0000   0.0000
1  0.25  0.3270   0.5005
2  0.50  0.3320   0.8612
3  0.75  0.3423   1.0750
4  1.00  1.0000   1.1442
